# Homework 5
Run the following cell to load `heart_processed.csv`, which has log-predictors from the [Heart Failure Clinical Records Dataset](https://archive.ics.uci.edu/ml/datasets/Heart%2Bfailure%2Bclinical%2Brecords) for predicting `DEATH_EVENT`.

In [1]:
import pandas as pd
import numpy as np

dataset = pd.read_csv("heart_processed.csv")
X = dataset.drop("DEATH_EVENT", axis=1)
y = dataset["DEATH_EVENT"]
X = X.iloc[:,1:]

Write a naive Bayes classifier with priors inferred from the dataset, and class-conditional densities inferred using `scipy.stats.gaussian_kde` with default bandwidth. Print the accuracy on the dataset.

In [2]:
classes = np.unique(y)
print ('There are %d classes:' % len(classes), classes)

There are 2 classes: [0 1]


In [3]:
from scipy import stats
param = dict()  # we save them in this dictionary
data = X.values.copy()
for k in classes:
    members = (y == k) # boolean array
    num = members.sum()    # True:1, False:0
    prior = num / y.size
    X_= data[members,:]
    feature_kde=[]
    for feature_i in range(data.shape[1]):
        xx=X_[:,feature_i]
        feature_kde_ = stats.gaussian_kde(xx)
        feature_kde.append(feature_kde_)
    param[k]=[num,prior,feature_kde]


In [4]:
# init predicted values
k_pred = -1 * np.ones(y.size)

# evaluate posterior for each point and find maximum
for i in range(y.size):
    pmax, kmax = -1, None   # initialize to nonsense values
    for k in classes:
        num, prior, kde = param[k]
        data = X.values.copy()[i,:]
        
        X_ = 1
        for l in range(len(kde)):
            feature_kde = kde[l]
            X_ = X_ * feature_kde.evaluate(data[l])
        p = prior * X_
        if p > pmax:
            pmax = p
            kmax = k
    k_pred[i] = kmax

print("Accuracy= ", np.sum(k_pred == y) / len(y))

Accuracy=  0.7892976588628763
